# 1.建模长序列的问题

1.RNN模型的介绍：<br>
（1）RNN模型是一种将前序步骤的输出作为当前步骤的输入的神经网络，这种特性使得RNN适用于序列数据（比如文本）；<br>

（2）RNN是一种encoder-decoder模型，当输入的文本被送进encoder之后，会进行如下的处理过程：<br>
encoder会在每一个step都更新自己的隐藏状态（hidden state），这个隐藏状态实际上那个就是hidden layers里的值，将整个输入序列的信息都传入最终的一个隐藏状态。这个部分的作用就是将输入的整个文本处理进一个hidden state，也叫做记忆单元（memory cell）。<br>

decoder就是将这一个最终的hidden state作为输入，将其用于生成输出的序列，生成的过程是一次只输出一个token的，也就是token by token的。这部分的作用就是利用encoder阶段得到的hidden state生成输出。<br>

（3）RNN的缺陷：<br>
RNN在decoder处理的阶段，无法直接访问encoder中的前序隐藏状态，而是只能依赖于encoder中得到的最终那一个hidden state（其中封装了输入序列中的所有相关信息）。这就会导致丢失上下文的信息，特别是在复杂的句子中，词与词之间的依赖关系跨越的距离非常大的情况下。<br>

2.在LLM出来之前，没有注意力机制的模型有哪些问题？<br>
正如上面介绍的RNN模型的缺陷：丢失上下文信息和无法建立长程依赖。以文本翻译为例，RNN这种逐词翻译的模型的效果就会很差，因为它在decoder阶段无法保留上下文信息和语言的语法结构。<br>

# 2.通过注意力机制捕捉数据依赖

1.Bahdanau attention mechanism：<br>
这是在2014年提出的RNN的改进方法，主要的改进之处在于：<br>
decoder可以在每一个decoding step，有选择性地去access输入序列中的任意部分token，并且在access的时候，可以通过注意力权重/分数（attention weights/scores），对输入序列中的不同token进行重要性衡量，进而对生成输出的token产生不同的重要性影响。<br>

2.2017年提出的transformer就是在bahdanau注意力机制的基础上改进的，使用了self-attention机制。<br>

3.Self-attention mechanism：就是在计算输入序列的表示（representation）的时候，允许输入序列中的每个位置都能够关注当前相同序列中的其他任意位置。<br>

# 3.使用自注意力关注输入中的不同部分

1.self-attention中的self指的是在一个输入序列中，通过将不同的位置关联起来，进而计算得到注意力权重。它可以评估和学习输入中各个部分之间的关系和依赖性。<br>

自注意力与传统的注意力机制不一样，传统的注意力机制是关注两个不同序列中的元素之间的关系，就比如上面提到的bahdanau注意力机制就是关注的输出序列中的某个元素和输入序列中所有元素之间的关系。<br>

## 3.1 一个不包含可训练参数的简化版自注意力机制

1.自注意力机制self-attention的目标就是针对输入序列中的每一个元素，计算它的上下文向量(context vector)，这个向量就包括了该元素与序列中其他所有元素之间的关系信息。这里的context vector可以理解为一个enriched embedding vector，也就是tokenization和embedding之后的加强版embedding。<br>

2.实现self-attention的第一步，也就是计算得到context vector的第一步。即计算注意力分数/权重：<br>
    （1）将输入序列中当前需要计算context vector的元素，作为query，将query的embedding vector，分别与输入序列中的每一个元素的embedding vector做点积(dot product)，包括当前元素本身也要计算，得到注意力分数；<br>
- 使用点积的愿意：点积的结果是一个常数，方便作为权重；点积也是一个用于衡量两个向量之间的相似度的方法，能够量化两个向量之间的对齐程度，结果越大表明两个向量之间越相似；在自注意力机制中，序列中元素之间的embedding vector的点积结果就表明了它们之间互相关注的程度，点积结果越大，表明两个元素之间的相似性越高。<br>

（2）在得到的注意力分数的基础上，应用归一化(normalization)方法对其进行归一化，得到最终的注意力权重（和为1），通常使用的是Softmax函数。<br>
- 归一化的目的就是让注意力权重的为一，这对LLM的解释性和训练过程中的训练稳定性有帮助；

# 4.包含可训练权重的自注意力机制

1.自注意力(self-attention)又叫做缩放点积注意力(scaled dot-product attention)。

2.可训练权重（trainable weights）指的是新引入的 3 个可训练的权重矩阵$W_q, W_k, W_v$。作用是将输入序列中每个token的embedding映射为query, key, value三个向量。

3.这里引入的query, key, value分别是什么含义？
这里引入的三个权重矩阵，是参考了信息检索和数据库领域的概念：使用相似的概念进行存储、搜索和检索信息。
- query：类似于数据中的查询语句，在自注意力机制中用于探测输入序列中的其他部分，以进一步确定对它们各自的关注程度（就是后面要得到的注意力权重）。
- key: 类似于数据库中用于indexing 和 searching的键。在这里是用来与query匹配的。
- value：就是数据库中的键值对的值。在这里代表的就是输入序列中token的实际值。

4.引入可训练权重之后，在上一节简化版自注意力计算context vector的三个步骤的基础上，有以下新的改动：

（1）__投影得到querys, keys, values__：
- 初始化可训练权重矩阵$W_q, W_k, W_v$：这里权重矩阵的shape是`（embedding_size, projection_size）`。这里projection_size就是会将每个token的embedding_size变成新的值，在GPT系列模型中，一般是让projection_size = embedding_size
- 通过矩阵乘法得到querys, keys, values： 这里就是使用权重矩阵与输入序列的token embedding矩阵之间进行矩阵乘法，得到投影后的query, keys, values矩阵。

（2）**计算注意力分数**：原来是使用输入序列中元素的token embedding之间进行点积，现在是使用（1）中得到的querys与keys之间进行矩阵相乘（内部实际上就是每一个元素query vector，分别与其他所有元素的key vector之间计算点积）。

（3）**归一化的得到注意力权重**：在原来只需要应用softmax()的基础上，添加了一个scale缩放操作，就是将每一个元素的注意力分数向量，除以 $\sqrt{d_k}$，这里的$d_k$就是keys的embedding_size，也就是初始化权重矩阵的时候的projection_size。
- 缩放操作的目的是避免当投影的维度太大，导致注意力分数向量中的数值太大，softmax函数在趋于无穷大的地方，梯度为0，因此在训练过程中，梯度反向传播的时可能会导致梯度消失的问题。

（4）__计算context vectors__：这里与简化版的使用注意力权重与token embedding之间加权求和不一样，这里是将注意力权重与每个token的value进行加权求和，得到最终的context vectors。